In [30]:
import CVIs
import helper
import methods 

In [34]:
import inspect

functs = [f for _, f in inspect.getmembers(methods, inspect.isfunction)]

print(functs)

for fn in functs:
    vars = fn.__code__.co_varnames
    print(vars)


[<function CAL at 0x12bc0f040>, <function K_means at 0x12b0b4a60>, <function PCKM at 0x12b0dd790>, <function euclidean_distances at 0x12118aee0>, <function fcluster at 0x12b0dcc10>, <function linkage at 0x12b0d7af0>, <function single_linkage at 0x12bc0fca0>]
('ML', 'CL')
('data', 'K')
('K', 'ML', 'CL')
('X', 'Y', 'Y_norm_squared', 'squared', 'X_norm_squared', 'original_shape')
('Z', 't', 'criterion', 'depth', 'R', 'monocrit', 'n', 'T')
('y', 'method', 'metric', 'optimal_ordering', 'n', 'method_code', 'result')
('data', 'dis_mat', 'X', 'Z', 'max_jump_ind', 'max_dis')
['CAL', 'KMeans', 'K_means', 'PCKM', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'euclidean_distances', 'fcluster', 'linkage', 'np', 'single_linkage', 'ssd']
